In [1]:
from pyCode.AuxDecays import DecayList,Decay
from pyCode.component import Component
from pyCode.EqFunctions import T as Tfunc
from pyCode.EqFunctions import gSTAR

INFO:pyCode.AuxFuncs:Loading aux functions. Ignoring BSM corrections to g* and g*_S


In [2]:
import numpy as np
import sympy as sp

In [3]:
lamb = 0.17
decays = DecayList()
decayToDM = Decay(instate='Mediator',fstates=['DM','radiation'],br=1.)
decays.addDecay(decayToDM)
decays.Xfraction = 0.1
decays.width = 2.49e-15*(lamb/4.3e-7)**2
dofDM = -2
dofMed = 6
dm = Component(label='DM',Type='thermal',dof=dofDM, mass=500.)
mediator = Component(label='Mediator',Type='thermal',dof=dofMed, mass=510.,decays=decays)
components = [dm,mediator]

In [4]:
#Store the number of components:
nComp = len(components)

#Define variables:        
N = np.array(sp.symbols('N:%d'%nComp))
R = np.array(sp.symbols('R:%d'%nComp))
NS = sp.symbols('N_S')
x = sp.symbols('x')
T = Tfunc(x,NS)

n = np.array([sp.exp(Ni) for Ni in N])

#Planck constant:
MP = sp.symbols('M_P')

#Current energy densities:
rho = n*R

#Compute equilibrium densities:
neq = np.array([comp.nEQ(T) for comp in components])

#Compute ratio of equilibrium densities
#(helps with numerical instabilities)
#rNeq[i,j] = neq[i]/neq[j]
rNeq = np.array([[compi.rNeq(T,compj) if compi.active and compj.active else 0. for compj in components] 
                 for compi in components])

#Dictionary with label:index mapping:
labelsDict = dict([[comp.label,i] for i,comp in enumerate(components)])
isActive = np.array([comp.active for comp in components])
        
#Compute Hubble factor:
rhoTot = np.sum(rho,where=isActive,initial=0)
rhoRad = (sp.pi**2/30)*gSTAR(T)*T**4  # thermal bath's energy density    
rho = rhoRad+rhoTot
H = sp.sqrt(8*sp.pi*rho/3)/MP
 
#Effective equilibrium densities and BRs:
NXth = np.array([comp.getNXTh(T,n,rNeq,labelsDict) for comp in components])
widths = np.array([comp.width(T) for comp in components])
masses = np.array([comp.mass(T) for comp in components])
BRX = np.array([comp.getBRX(T) for comp in components])
sigmaV = np.array([comp.getSIGV(T) for comp in components])

dNS = np.sum(isActive*BRX*widths*masses*(n-NXth))*sp.exp(3.*x - NS)/(H*T)

In [5]:
RHS = -3*n
#Decay term:
RHS -= widths*masses*n/(H*R)
#Inverse decay term:
RHS += widths*masses*NXth/(H*R) #NXth should be finite if i -> j +..
#Annihilation term:            
RHS += sigmaV*(neq - n)*(neq + n)/H
dN = sp.sympify(np.zeros(nComp)).as_mutable()
for i,rhs in enumerate(RHS):
    if isActive[i]:
        dN[i] = RHS[i]/n[i]

In [6]:
RHS = sp.sympify(np.zeros(nComp)).as_mutable()
#Derivatives for the rho/n variables (only for thermal components):
for i,comp in enumerate(components):
    if not isActive[i]:
        continue
    RHS[i] = -3.*n[i]*comp.Pn(T,R[i])  #Cooling term

dR = sp.sympify(np.zeros(nComp)).as_mutable()
for i,rhs in enumerate(RHS):
    if isActive[i]:
        dR[i] = rhs/n[i]

In [7]:
dy = np.hstack((dN,dR,[dNS]))
yv = np.hstack((N,R,[NS]))
print(yv)

[N0 N1 R0 R1 N_S]


In [8]:
fff = sp.lambdify([x,yv],dy, modules=[{'M_P' : 1.22e19},'numpy','sympy'])

In [22]:
type(fff(1.,[10.,5.,500.,500.,2.]))

list

In [15]:
jac = sp.Matrix(dy).jacobian(yv).tolist()
jacf = sp.lambdify([x,yv],jac,modules=[{'M_P' : 1.22e19},'numpy','sympy'])

In [23]:
np.array(jac).shape

(5, 5)

In [12]:
jf = sp.lambdify([x,yv],jac[4,0],modules=[{'M_P' : 1.22e19},'numpy','sympy'])

In [13]:
jf(1.,[10.,5.,500.,500.,2.])

-0.00162418998337005*M_P

In [14]:
dy[1]

(0.0496215657111952*sqrt(6)*M_P*rNeq(T(x, N_S), 510.0, 6, 500.0, -2)*exp(N0)/(sqrt(pi)*R1*sqrt(R0*exp(N0) + R1*exp(N1) + pi**2*T(x, N_S)**4*gSTAR(T(x, N_S))/30)) - 0.0496215657111952*sqrt(6)*M_P*exp(N1)/(sqrt(pi)*R1*sqrt(R0*exp(N0) + R1*exp(N1) + pi**2*T(x, N_S)**4*gSTAR(T(x, N_S))/30)) - 3*exp(N1))*exp(-N1)